In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload the ZIP file


Saving archive.zip to archive.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import zipfile
import os

zip_path = "/content/archive.zip"  # Change to your file name
extract_folder = "/content/dataset"  # Destination folder

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Files extracted to:", extract_folder)


Files extracted to: /content/dataset


In [4]:
# Define Dataset Paths
dataset_dir = "/content/dataset/split_ttv_dataset_type_of_plants"
train_dir = os.path.join(dataset_dir, "Train_Set_Folder")
val_dir = os.path.join(dataset_dir, "Validation_Set_Folder")
test_dir = os.path.join(dataset_dir, "Test_Set_Folder")


In [5]:
img_size = 128  # Resize images to 128x128
batch_size = 32

In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [7]:
gen_train = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 23972 images belonging to 30 classes.


In [8]:
gen_val = datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 3030 images belonging to 30 classes.


In [9]:
class_names = list(gen_train.class_indices.keys())

In [10]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(class_names), activation='softmax')
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train Model
history = model.fit(
    gen_train,
    validation_data=gen_val,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 166s 210ms/step - accuracy: 0.0792 - loss: 3.9927 - val_accuracy: 0.1178 - val_loss: 2.9658 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 152s 202ms/step - accuracy: 0.1147 - loss: 2.9954 - val_accuracy: 0.2116 - val_loss: 2.5375 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 150s 200ms/step - accuracy: 0.1641 - loss: 2.7617 - val_accuracy: 0.2545 - val_loss: 2.4801 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 149s 199ms/step - accuracy: 0.1868 - loss: 2.6315 - val_accuracy: 0.2957 - val_loss: 2.2499 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 154s 205ms/step - accuracy: 0.2209 - loss: 2.4969 - val_accuracy: 0.2337 - val_loss: 2.6019 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 149s 199ms/step - accuracy: 0.2468 - loss: 2.3995 - val_accuracy: 0.2284 - val_loss: 3.2508 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 202s 199ms/step - accura

In [11]:
model.save("crop_classification_cnn.h5")